In [1]:
import datetime as _dt
import requests as _rq
import pandas as _pd


API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api"
HEADERS = {"User-Agent": "EOD-Fundamentals-Client/2.0"}


def _get_json(url: str):
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _annual_eps_dict(fund: dict) -> dict[int, float]:
    annual_raw = fund.get("Earnings", {}).get("Annual", {})
    eps_year = {}
    for date_key, rec in annual_raw.items():
        eps = rec.get("epsActual")
        if eps is not None:
            eps_year[int(date_key[:4])] = eps
    return eps_year


def _sum_quarterly_eps(earnings_hist: list[dict]) -> dict[int, float]:
    eps_q = {}
    for rec in earnings_hist:
        eps = rec.get("epsActual")
        d = _pd.to_datetime(rec.get("date") or rec.get("reportDate"), errors="coerce")
        if _pd.notna(d) and eps is not None:
            eps_q.setdefault(d.year, []).append(eps)
    return {y: sum(v) for y, v in eps_q.items() if len(v) == 4 and sum(v) != 0}


def _five_year_pe_series(df_price: _pd.DataFrame, eps_year: dict[int, float]) -> list[float]:
    years = sorted(eps_year)[-5:]
    pe = []
    for y in years:
        yr_px = df_price[df_price["date"].dt.year == y]
        if yr_px.empty:
            continue
        eps = eps_year[y]
        if eps == 0:
            continue
        pe.append(yr_px.iloc[-1]["adjusted_close"] / eps)
    return pe


def _ttm_eps(earnings_hist: list[dict], today: _dt.date) -> float | None:
    hist_sorted = sorted(
        (
            (rec.get("epsActual"), _pd.to_datetime(rec.get("reportDate") or rec.get("date"), errors="coerce"))
            for rec in earnings_hist
            if rec.get("epsActual") is not None
        ),
        key=lambda x: x[1],
        reverse=True,
    )
    eps_vals = []
    for eps, d in hist_sorted:
        if d is not None and d.date() <= today:
            eps_vals.append(eps)
            if len(eps_vals) == 4:
                break
    return sum(eps_vals) if len(eps_vals) == 4 else None


def CurrentPrice(ticker):
    url = f"{BASE_URL}/real-time/{ticker}?api_token={API_TOKEN}&fmt=json"
    d = _get_json(url)
    return d.get("close") or d.get("price") or d.get("lastPrice")


def ATH(ticker):
    today = _dt.date.today().strftime("%Y-%m-%d")
    url = f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}&from=1900-01-01&to={today}&adjusted=1&fmt=json"
    df = _pd.DataFrame(_get_json(url))
    return None if df.empty else df["adjusted_close"].max()


def Fundamentals(ticker) -> _pd.DataFrame:
    fund = _get_json(f"{BASE_URL}/fundamentals/{ticker}?api_token={API_TOKEN}&fmt=json")

    hi = fund.get("Highlights", {})
    pe_curr_api = hi.get("PERatio")
    eps_ttm = hi.get("DilutedEpsTTM")

    hist_raw = fund.get("Earnings", {}).get("History", {})
    earnings_hist = list(hist_raw.values()) if isinstance(hist_raw, dict) else (hist_raw or [])

    eps_year = _annual_eps_dict(fund)
    if not eps_year:
        eps_year = _sum_quarterly_eps(earnings_hist)

    today_dt = _dt.datetime.now()
    to_date = today_dt.strftime("%Y-%m-%d")
    from_date = (today_dt.replace(year=today_dt.year - 6)).strftime("%Y-%m-%d")
    url_hist = f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}&from={from_date}&to={to_date}&adjusted=1&fmt=json"
    dfp = _pd.DataFrame(_get_json(url_hist))
    if dfp.empty:
        dfp = _pd.DataFrame(columns=["date", "adjusted_close"])
    dfp["date"] = _pd.to_datetime(dfp["date"])

    pe_series = _five_year_pe_series(dfp, eps_year)
    avg_pe5y = sum(pe_series) / len(pe_series) if len(pe_series) >= 2 else None

    pe_curr = pe_curr_api
    if pe_curr is None:
        if eps_ttm and eps_ttm != 0:
            price_now = dfp.iloc[-1]["adjusted_close"] if not dfp.empty else CurrentPrice(ticker)
            pe_curr = price_now / eps_ttm

    today = today_dt.date()
    fut_dates = [
        _pd.to_datetime(r.get("reportDate"), errors="coerce").date()
        for r in earnings_hist if r.get("reportDate")
    ]
    fut_dates = [d for d in fut_dates if d and d > today]
    next_er = min(fut_dates) if fut_dates else None

    high_5y = dfp["adjusted_close"].max(skipna=True)
    cutoff = _pd.Timestamp(today_dt) - _pd.Timedelta(days=180)
    high_6m = dfp.loc[dfp["date"] > cutoff, "adjusted_close"].max(skipna=True)

    eps_est = hi.get("EPSEstimateNextYear")
    val_ttm = avg_pe5y * eps_ttm if avg_pe5y and eps_ttm else None
    val_est = avg_pe5y * eps_est if avg_pe5y and eps_est else None

    row = {
        "Ticker": ticker,
        "Current Price": CurrentPrice(ticker),
        "All‑Time High": ATH(ticker),
        "5‑Year High": high_5y,
        "6‑Month High": high_6m,
        "Next Earnings Date": next_er,
        "Current P/E": pe_curr,
        "Avg 5‑Year P/E": avg_pe5y,
        "Avg P/E × TTM EPS": val_ttm,
        "Avg P/E × Forecast EPS": val_est,
    }
    return _pd.DataFrame([row])


def FundamentalsBuilder(tickers):
    return _pd.concat([Fundamentals(t) for t in tickers], ignore_index=True)


def SaveToCSV(df, fname):
    df.to_csv(fname, index=False, encoding='utf-8-sig')
    print(f"Saved {len(df)} rows → {fname}")


if __name__ == "__main__":
    raw1 = ['aapl', 'msft', 'amd']
    raw = [
        "NET", "CRS", "ORLY", "DOCN", "TEM", "VST", "CVS", "TER", "RIVN", "NTGR",
        "SPOT", "UBER", "SNOW", "ACHR", "ASAN", "RKLB", "DPZ", "LYFT", "NEGG", "MSFT",
        "AAPL", "ARM", "AMZN", "META", "NFLX", "NVDA", "GOOG", "RDFN", "MDB", "Z",
        "INTC", "AMD", "AFRM", "COIN", "TSLA", "HOOD", "CAVA", "F", "GM", "SMR", "U",
        "RDDT", "SHOP", "SOFI"
    ]
    tickers = [f"{t}.US" for t in raw1]
    df_out = FundamentalsBuilder(tickers)
    SaveToCSV(df_out, "fundamentals_eod.csv")


Saved 3 rows → fundamentals_eod.csv


In [2]:
display(df_out)

,Ticker,Current Price,All‑Time High,5‑Year High,6‑Month High,Next Earnings Date,Current P/E,Avg 5‑Year P/E,Avg P/E × TTM EPS,Avg P/E × Forecast EPS
0,aapl.US,207.8500,258.7355,258.7355,258.7355,2025-05-01,33.2190,41.520677,261.580262,330.940552
1,msft.US,387.9100,464.8544,464.8544,453.5506,2025-04-30,31.5499,40.048472,497.402018,600.751104
2,amd.US,94.8398,211.3800,211.3800,149.8200,2025-05-06,96.6450,40.424603,40.424603,242.648680
